In [1]:
from sqlalchemy import create_engine
from MyCreds.mycreds import Capstone_AWS_SG, MapBox
import geopandas as gpd
import plotly.express as px

In [2]:
def get_animation_df():
    sql = """select mp.zone_id, mp."SUBZONE_N", mp.geometry,  avg(lf.resale_price) as avg_resale, min(lf.resale_price) as min_resale, max(lf.resale_price) as max_resale, count(distinct(transaction_id)) as count_resale, extract(year from lf.month) as transaction_year
from master_plan_2019_subzone_boundary as mp
         join resale_location_features as lf
              on ST_Contains(mp.geometry, lf.geometry)

group by transaction_year, mp.zone_id, mp."SUBZONE_N", mp.geometry;"""

    #where extract(year from lf.month) > 2010

    engine = create_engine(f'postgresql+psycopg2://{Capstone_AWS_SG.username}:{Capstone_AWS_SG.password}@{Capstone_AWS_SG.host}/Capstone', echo=False)

    with engine.connect() as cnxn:
        locs_in_plan = gpd.read_postgis(sql, cnxn, geom_col='geometry')

    return locs_in_plan

In [ ]:
df = get_animation_df()